In [9]:
import numpy as np
import networkx as nx
import sys
sys.path.append("/Users/nando/Repos/DCOP/py_dcop/src/")

from core.dcop_instance import DCOPInstance
from algorithms.ccg_maxsum import transform_dcop_instance_to_ccg

data_path = '/Users/nando/Repos/DCOP/py_dcop/data/'
dcopIstance = DCOPInstance(data_path + 'binary.json')
G = transform_dcop_instance_to_ccg(dcopIstance)

Running /Users/nando/Repos/DCOP/py_dcop/third_parties/wcsp/build/bin/wcsp -k -g /var/folders/q4/003xq7117nnfy9_cth24pwk80000gs/T/tmp7vijws_j


In [59]:
# Associates variables to CCG nodes
var_to_ccg_nodes = { vname : [] for vname in dcopIstance.variables}
for n, d in G.nodes(data=True):
    if 'variable' in d:
        var_to_ccg_nodes[d['variable']].append(n)

# Example to retrieve node from variable
for n in var_to_ccg_nodes['v0']:
    print(G.node[n])

{'weight': 30.0, 'type': 'decision', 'variable': 'v0', 'rank': 1}
{'weight': 22.0, 'type': 'decision', 'variable': 'v0', 'rank': 2}


In [72]:
## Partition the nodes among agents:
processed_nodes = []

G_agts = {vname: nx.Graph() for vname in dcopIstance.variables}

for v in dcopIstance.variables:
    for n in var_to_ccg_nodes[v]:
        G_agts[v].add_node(n, attr_dict=G.node[n])
        processed_nodes.append(n)
        
    for n in var_to_ccg_nodes[v]:
        ngbs_n = G.neighbors(n)
        for m in ngbs_n:
            if m not in processed_nodes:
                G_agts[v].add_node(m, attr_dict=G.node[n])
                processed_nodes.append(m)

In [79]:
print(sorted(G_agts['v0'].nodes))
print(sorted(G_agts['v1'].nodes))
print(sorted(G_agts['v2'].nodes))

['1', '10', '11', '12', '13', '15', '17', '19', '2', '21', '23', '25', '27', '29', '31', '33', '5', '6', '9']
['16', '20', '22', '28', '3', '30', '34', '35', '4']
['14', '18', '24', '26', '32', '36', '37', '7', '8']


In [91]:
## Partition edges among agents
processed_edges = []
for v in dcopIstance.variables:
    #print(nx.edges(G, G_agts[v].nodes))
    tmp = []
    for n in G_agts[v].nodes:
        for e in nx.edges(G, n):
            if (e[0], e[1]) in processed_edges: continue
            if (e[1], e[0]) in processed_edges: continue
            processed_edges.append((e[0], e[1]))
            processed_edges.append((e[1], e[0]))
            tmp.append(e)
    G_agts[v].add_edges_from(tmp)

In [93]:
# Finally Add nodes of connecting edges:
print(G_agts['v0'].edges)
print(G_agts['v1'].edges)
print(G_agts['v2'].edges)

[('1', '6'), ('1', '10'), ('1', '11'), ('1', '12'), ('1', '13'), ('1', '15'), ('1', '21'), ('1', '23'), ('1', '25'), ('1', '27'), ('1', '2'), ('2', '5'), ('2', '9'), ('2', '11'), ('2', '12'), ('2', '13'), ('2', '17'), ('2', '19'), ('2', '29'), ('2', '31'), ('2', '33'), ('6', '5'), ('10', '9'), ('11', '3'), ('12', '4'), ('13', '7'), ('15', '14'), ('21', '20'), ('23', '22'), ('25', '24'), ('27', '26'), ('5', '3'), ('5', '4'), ('9', '7'), ('9', '8'), ('17', '16'), ('19', '18'), ('29', '28'), ('31', '30'), ('33', '32'), ('3', '16'), ('3', '20'), ('3', '28'), ('3', '35'), ('3', '4'), ('4', '16'), ('4', '22'), ('4', '30'), ('4', '34')]
[('35', '34')]
[('7', '14'), ('7', '18'), ('7', '24'), ('7', '32'), ('7', '37'), ('7', '8'), ('8', '14'), ('8', '18'), ('8', '26'), ('8', '36'), ('37', '36')]


In [97]:
for v in dcopIstance.variables:
    for (n1, n2) in G_agts[v].edges:
        G_agts[v].add_node(n2, attr_dict=G.node[n2])